In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from torchvision import datasets, transforms
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import glob
import random
import cv2
import imutils
from skimage import color
from skimage.util.dtype import convert
import import_ipynb #pip install import-ipynb

from data import ImageNet, VisionDataset
from CUnet import CUNet
from eccv16 import eccv16

importing Jupyter notebook from data.ipynb
importing Jupyter notebook from CUnet.ipynb
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 128, 128])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 2, 256, 256])
importing Jupyter notebook from eccv16.ipynb


## Define models and data loaders

In [2]:
cu_model = CUNet()
ecc_model = eccv16()

In [3]:
trans = transforms.Compose([transforms.ToTensor()])
ImageNet_train_dataset = ImageNet(root="./imagenet/train", transform=trans, size=1000)
ImageNet_eval_dataset = ImageNet(root="./imagenet/val", transform=trans, size=1000)

In [4]:
train_dataloader_imagenet = DataLoader(ImageNet_train_dataset, batch_size=32, shuffle=True)
eval_dataloader_imagenet = DataLoader(ImageNet_eval_dataset, batch_size=32, shuffle=True)

## Define Hyper params

In [5]:
EPOCHS = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.SGD(cu_model.parameters(), lr = 0.1) #change as needed
loss_function = torch.nn.MSELoss() #change as needed

## Define Train and eval functions

In [6]:
def train_model(dataloader, model, device, optimizer, loss_function, epochs=1):
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for batch_index,(inputs, expected) in enumerate(dataloader):
            optimizer.zero_grad() 
            inputs = inputs.to(device)
            expected = expected.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, expected)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*len(expected)
        filename = f'model/{type(model).__name__}_{epoch}_{train_loss}.pth'
        torch.save(model.state_dict(), filename)

In [7]:
def eval_model(dataloader, model, device, optimizer, loss_function, epochs=1):
    model.eval()
    val_loss = 0
    for batch_index,(inputs, expected) in enumerate(dataloader):
        # inputs = torchvision.transforms.ToTensor(train_dataloader_imagenet.resize_image(lightness))
        # expected = torchvision.transforms.ToTensor(train_dataloader_imagenet.resize_image(colors))
        # inputs.type(torch.float32)
        # expected.type(torch.float32)
        print(type(inputs))
        inputs = inputs.to(device)
        expected = expected.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, expected)
        val_loss += loss.item()*len(expected)
    avg_val_loss = val_loss/len(eval_dataloader_imagenet)

In [8]:
def name_to_epoch(name):
    _, a = name.split("/")
    _, epoch, _ = name.split("_") #name, epoch, avg_l.pth
    return(int(epoch))

In [9]:
train_model(train_dataloader_imagenet, cu_model, device, optimizer, loss_function, 1)

In [10]:
train_model(train_dataloader_imagenet, ecc_model, device, optimizer, loss_function, 1)